# 技能路径匹配模型训练示例

本笔记本展示了如何使用技能路径匹配框架训练模型。

In [ ]:
# 导入必要的库
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

# 添加项目根目录到路径
sys.path.append('..')

# 导入自定义模块
from src.models.matching_model import DeepMatchingModel, DualEncoderMatchingModel, SkillPathAwareMatchingModel
from src.utils.data_utils import load_data, extract_text_features, combine_features, SkillMatchingDataset, create_data_loaders
from src.training.trainer import Trainer
from src.evaluation.metrics import calculate_classification_metrics, calculate_ranking_metrics
from src.utils.visualization import plot_learning_curves, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

## 1. 数据准备

首先，我们需要准备训练数据。在这个示例中，我们将使用模拟数据。

In [ ]:
# 创建模拟数据
def create_synthetic_data(n_samples=1000, resume_dim=100, job_dim=80, skill_graph_dim=50):
    # 创建随机特征
    resume_features = np.random.randn(n_samples, resume_dim)
    job_features = np.random.randn(n_samples, job_dim)
    skill_graph_features = np.random.randn(n_samples, skill_graph_dim)
    
    # 创建随机标签（0或1）
    labels = np.random.randint(0, 2, size=(n_samples, 1)).astype(np.float32)
    
    return resume_features, job_features, skill_graph_features, labels

# 生成模拟数据
resume_features, job_features, skill_graph_features, labels = create_synthetic_data()

# 查看数据形状
print(f"简历特征形状: {resume_features.shape}")
print(f"职位特征形状: {job_features.shape}")
print(f"技能图特征形状: {skill_graph_features.shape}")
print(f"标签形状: {labels.shape}")

## 2. 创建数据集和数据加载器

In [ ]:
# 创建数据集
dataset = SkillMatchingDataset(
    resume_features=resume_features,
    job_features=job_features,
    skill_graph_features=skill_graph_features,
    labels=labels
)

# 创建数据加载器
train_loader, val_loader, test_loader = create_data_loaders(
    dataset=dataset,
    batch_size=32,
    train_ratio=0.7,
    val_ratio=0.15,
    random_seed=42
)

print(f"训练集大小: {len(train_loader.dataset)}")
print(f"验证集大小: {len(val_loader.dataset)}")
print(f"测试集大小: {len(test_loader.dataset)}")

## 3. 定义模型

In [ ]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 定义模型参数
resume_dim = resume_features.shape[1]
job_dim = job_features.shape[1]
skill_graph_dim = skill_graph_features.shape[1]

# 创建模型
model = SkillPathAwareMatchingModel(
    resume_dim=resume_dim,
    job_dim=job_dim,
    skill_graph_dim=skill_graph_dim,
    hidden_dim=128,
    output_dim=64
)

# 打印模型结构
print(model)

## 4. 定义损失函数和优化器

In [ ]:
# 定义损失函数
criterion = nn.BCELoss()

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 定义学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

## 5. 定义评估指标

In [ ]:
# 定义评估指标函数
def calculate_metrics(y_true, y_pred):
    # 分类指标
    classification_metrics = calculate_classification_metrics(y_true, y_pred)
    
    # 排序指标
    ranking_metrics = calculate_ranking_metrics(y_true, y_pred)
    
    # 合并指标
    metrics = {}
    metrics.update(classification_metrics)
    metrics.update(ranking_metrics)
    
    return metrics

# 创建指标函数字典
metric_functions = {
    'accuracy': lambda y_true, y_pred: calculate_classification_metrics(y_true, y_pred)['accuracy'],
    'precision': lambda y_true, y_pred: calculate_classification_metrics(y_true, y_pred)['precision'],
    'recall': lambda y_true, y_pred: calculate_classification_metrics(y_true, y_pred)['recall'],
    'f1': lambda y_true, y_pred: calculate_classification_metrics(y_true, y_pred)['f1'],
    'auc': lambda y_true, y_pred: calculate_classification_metrics(y_true, y_pred)['auc']
}

## 6. 训练模型

In [ ]:
# 创建训练器
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    scheduler=scheduler,
    early_stopping_patience=10,
    model_save_path='../models/skill_path_model.pt'
)

# 训练模型
history = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=30,
    metric_functions=metric_functions,
    verbose=True,
    log_interval=1
)

## 7. 可视化训练过程

In [ ]:
# 准备绘图数据
train_metrics = {
    'loss': history['train_loss'],
    'accuracy': history['train_accuracy'],
    'precision': history['train_precision'],
    'recall': history['train_recall'],
    'f1': history['train_f1'],
    'auc': history['train_auc']
}

val_metrics = {
    'loss': history['val_loss'],
    'accuracy': history['val_accuracy'],
    'precision': history['val_precision'],
    'recall': history['val_recall'],
    'f1': history['val_f1'],
    'auc': history['val_auc']
}

# 绘制学习曲线
plot_learning_curves(
    train_metrics=train_metrics,
    val_metrics=val_metrics,
    figsize=(15, 10),
    title='模型训练学习曲线'
)

## 8. 在测试集上评估模型

In [ ]:
# 在测试集上评估模型
test_metrics = trainer.test(test_loader, metric_functions)

# 打印测试结果
print("测试集评估结果:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.6f}")

## 9. 可视化预测结果

In [ ]:
# 在测试集上进行预测
predictions = trainer.predict(test_loader)

# 获取真实标签
all_targets = []
for batch in test_loader:
    all_targets.append(batch['label'].numpy())
targets = np.concatenate(all_targets)

# 绘制混淆矩阵
plot_confusion_matrix(
    y_true=targets,
    y_pred=predictions,
    threshold=0.5,
    figsize=(8, 6),
    title='混淆矩阵'
)

# 绘制ROC曲线
plot_roc_curve(
    y_true=targets,
    y_pred=predictions,
    figsize=(8, 6),
    title='ROC曲线'
)

# 绘制精确率-召回率曲线
plot_precision_recall_curve(
    y_true=targets,
    y_pred=predictions,
    figsize=(8, 6),
    title='精确率-召回率曲线'
)

## 10. 模型应用示例

In [ ]:
# 创建一些测试样本
test_resume_features = np.random.randn(5, resume_dim)
test_job_features = np.random.randn(5, job_dim)
test_skill_graph_features = np.random.randn(5, skill_graph_dim)

# 转换为PyTorch张量
test_resume_tensor = torch.FloatTensor(test_resume_features).to(device)
test_job_tensor = torch.FloatTensor(test_job_features).to(device)
test_skill_graph_tensor = torch.FloatTensor(test_skill_graph_features).to(device)

# 使用模型进行预测
model.eval()
with torch.no_grad():
    match_scores = model(test_resume_tensor, test_job_tensor, test_skill_graph_tensor)

# 打印匹配分数
print("匹配分数:")
for i, score in enumerate(match_scores.cpu().numpy()):
    print(f"样本 {i+1}: {score[0]:.6f}")

## 11. 总结

在这个笔记本中，我们演示了如何使用技能路径匹配框架训练模型。我们使用了模拟数据来训练一个技能路径感知的匹配模型，并在测试集上评估了模型的性能。

主要步骤包括：
1. 准备数据
2. 创建数据集和数据加载器
3. 定义模型
4. 定义损失函数和优化器
5. 定义评估指标
6. 训练模型
7. 可视化训练过程
8. 在测试集上评估模型
9. 可视化预测结果
10. 模型应用示例

这个框架可以扩展到真实数据上，只需要替换模拟数据的部分，使用实际的简历、职位和技能图数据即可。